Task: Predict Rainfall in Australia

Dataset: https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package

Algorithm: Random Forest Classifier

How to Run:



In [ ]:
import numpy as np #importing numpy with name np
import pandas as pd #importing pandas with name pd

In [ ]:
# Drop unused columns and columns and rows containing 'NA'

df = pd.read_csv("/content/weatherAUS.csv") #assigning data to the csv file
df.drop(['Sunshine', 'Evaporation', 'Cloud3pm', 'Cloud9am'], inplace=True, axis=1) #dropping columns that contain all NA
df = df.replace('NA', np.nan).dropna() #dropping any row that contains an NA in one of its elements
df.columns

In [ ]:
# Reformat Date column from 'yyyy-mm-dd' to 'mm-dd'

for date in df['Date']:
  date = date[5:]

  # if len(date) != 5:
  #   raise Exception("Date not acceptable")


In [ ]:
# convert all text in data to numeric value

text_columns = df.select_dtypes(include=[object]).columns
print('text_columns: ', text_columns)

for col in text_columns:
  unique_vals = df[col].unique()
  num = 0
  for val in unique_vals:
    df[col].replace(val, num, inplace=True)
    num += 1

# Save preprocessed data to new csv
df.to_csv('/content/weatherAUSProcessed.csv')
df.head()

In [ ]:
# print name of data type of each column to verify numeric values

for col in df.columns:
  print('column name: \t', col)
  print('data type: \t', df[col].dtypes)
  print('\n')

In [ ]:
from sklearn.model_selection import train_test_split

# train:test split of 80:20
# random_state ensures consistent split on each instance
# stratify forces equal representation of "yes" and "no" in y columns

# Without stratification
normal_split = train_test_split(X, y, test_size=0.20, random_state=1)

print("Normal X_train.shape:\t\t", normal_split[0].shape, "\nNormal X_test.shape:\t\t", normal_split[1].shape)


# With stratification
stratified_split = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)

print("\nStratified X_train.shape:\t", stratified_split[0].shape, "\nStratified X_test.shape:\t", stratified_split[1].shape)

y_counts = y.value_counts()
norm_counts = normal_split[3].value_counts()
strat_counts = stratified_split[3].value_counts()
print(f"\ny counts: \n{y_counts} \nNo:Yes ratio: \n{y_counts[1]/sum(y_counts)}")
print(f"\nnormal_split counts: \n{norm_counts} \nNo:Yes ratio: \n{norm_counts[1]/sum(norm_counts)}")
print(f"\nstratified_split counts: \n{strat_counts} \nNo:Yes ratio: \n{strat_counts[1]/sum(strat_counts)}")

Feature Scaling Not Needed

It's noticeable that most columns have unnormalized data. Though, this shouldn't be a problem for a randrom forest.

"CART, Random Forests, Gradient Boosted Decision Trees. These algorithms utilize rules (series of inequalities) and do not require normalization."
-- https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35

Random Forest Classifier Design:

Random Forests are designed to work with tabular data. Feature scaling is not necessary for an RF model. 

In [ ]:
# A few things to make testing and comparing designs easy
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix

class RFC_model:
  def __init__(self, name, model):
    self.name = name
    self.model = model
    self.trained_model = None
    self.y_pred = None
    self.y_test = None
    self.metrics = {"Accuracy": None,
                   "Precision": None,
                   "F1": None,
                   "Recall": None,
                   "Confusion": None}


  def train_test_model(self, X_train, X_test, y_train):
    self.trained_model = self.model.fit(X_train, y_train)
    self.y_pred = self.model.predict(X_test)

    return self.y_pred


  def score_model(self, y_test):
    if self.y_pred is None: return

    y1 = y_test
    y2 = self.y_pred
    self.metrics.update({"Accuracy": accuracy_score(y1, y2),
              "Precision": precision_score(y1, y2),
              "F1": f1_score(y1, y2),
              "Recall": recall_score(y1, y2),
              "Confusion": confusion_matrix(y1, y2)})
    
    return self.metrics


  def get_all(self):
    contents = (self.name, self.model, self.trained_model, 
                self.y_pred, self.y_test, self.metrics)
    return contents

In [ ]:
# Create a few versions of a Random Forest
# Two sets: one for each type of data split
from sklearn.ensemble import RandomForestClassifier

number_of_trees = [2, 5, 10, 20, 50, 100, 150, 200, 250]
models = []

for i, n in enumerate(number_of_trees):
  model = RandomForestClassifier(n_estimators=n, max_depth=3, random_state=0)
  models.append(RFC_model(f"model{i}" , model))
  print(f"model{i}")

In [ ]:
X_train, X_test, y_train, y_test = stratified_split

print("Stratified Split: ")
for model in models:
  model.train_test_model(X_train, X_test, y_train)
  model.score_model(y_test)
  print(model.metrics)
# for model in models:
#   y_pred = train_test_model(model, X_train, y_train, X_test)
#   scores = score_model(y_test, y_pred)
# print_scores(scores)